In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# read the excel file
data = pd.read_excel('coordinates.xlsx')

# extract x, y, and z coordinates
x = data['x']
y = data['y']
z = data['z']

# create a 3D figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# plot the scatter
ax.scatter(x, y, z)

# set labels for each axis
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

# show the plot
plt.show()


In [ ]:
import numpy as np
from scipy.spatial import Delaunay

# read the excel file
data = pd.read_excel('coordinates.xlsx')

# extract x, y, and z coordinates
x = data['x']
y = data['y']
z = data['z']

# create a Delaunay triangulation of the data
points = np.column_stack((x, y, z))
tri = Delaunay(points)

# plot the surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(points[:, 0], points[:, 1], points[:, 2], triangles=tri.simplices, cmap=plt.cm.Spectral)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Generate random points on a sphere
n_points = 3000
theta = np.random.uniform(0, np.pi, n_points)
phi = np.random.uniform(0, 2*np.pi, n_points)
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

# Plot the sphere with scattered points
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z)

# Define the quadratic function to fit
def quadratic_func(data, p, q, r, s, t, c):
    x, y = data
    return p*x**2 + q*y**2 + r*x*y + s*x + t*y + c

# Fit the quadratic function to the scattered points
data = (x, y)
p0 = [1, 1, 1, 1, 1, 1] # Initial guess for parameters
params, _ = curve_fit(quadratic_func, data, z, p0=p0)

# Evaluate the quadratic function at each point on the sphere
xx, yy = np.meshgrid(np.linspace(-1, 1, 50), np.linspace(-1, 1, 50))
zz = quadratic_func((xx, yy), *params)

# Calculate the mean curvature at each point on the sphere
dx = 2/(xx.shape[1]-1)
dy = 2/(yy.shape[0]-1)
x_grad, y_grad = np.gradient(zz, dx, dy)
x_grad2, y_grad2 = np.gradient(x_grad, dx, dy)
y_grad2, x_grad2 = np.gradient(y_grad, dy, dx)
mean_curvature = (x_grad2*y_grad**2 - 2*x_grad*y_grad*x_grad2 + y_grad2*x_grad**2) / ((x_grad**2 + y_grad**2 + 1e-10)**1.5)

# Plot the fitted quadratic function and mean curvature
ax.plot_surface(xx, yy, zz, alpha=0.5)
ax.scatter(x, y, quadratic_func(data, *params))
fig, ax = plt.subplots()
ax.imshow(mean_curvature, cmap='RdYlBu')
plt.show()


In [ ]:
mean_curvature.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pandas as pd

# Load data from Excel file
df = pd.read_excel('coordinates.xlsx')
x = df['x'].values
y = df['y'].values
z = df['z'].values

# Plot the scattered data
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z)

# Define the quadratic function to fit
def quadratic_func(data, p, q, r, s, t, c):
    x, y = data
    return p*x**2 + q*y**2 + r*x*y + s*x + t*y + c

# Fit the quadratic function to the scattered points
data = (x, y)
p0 = [1, 1, 1, 1, 1, 1] # Initial guess for parameters
params, _ = curve_fit(quadratic_func, data, z, p0=p0)

# Evaluate the quadratic function at each point on the sphere
xx, yy = np.meshgrid(np.linspace(min(x), max(x), 50), np.linspace(min(y), max(y), 50))
zz = quadratic_func((xx, yy), *params)

# Calculate the mean curvature at each point on the sphere
dx = 2/(xx.shape[1]-1)
dy = 2/(yy.shape[0]-1)
x_grad, y_grad = np.gradient(zz, dx, dy)
x_grad2, y_grad2 = np.gradient(x_grad, dx, dy)
y_grad2, x_grad2 = np.gradient(y_grad, dy, dx)
mean_curvature = (x_grad2*y_grad**2 - 2*x_grad*y_grad*x_grad2 + y_grad2*x_grad**2) / ((x_grad**2 + y_grad**2 + 1e-10)**1.5)

# Plot the fitted quadratic function and mean curvature
ax.plot_surface(xx, yy, zz, alpha=0.5)
ax.scatter(x, y, quadratic_func(data, *params))
fig, ax = plt.subplots()
ax.imshow(mean_curvature, cmap='RdYlBu')
plt.show()



In [ ]:
yy[0]

In [ ]:
xx[0]


In [ ]:
mean_curvature[0]

In [ ]:
import open3d as o3d

In [ ]:
from scipy.linalg import eig
from scipy.spatial.distance import pdist, squareform
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d

def curvature(tri, points):
    P = points[tri.simplices]
    du = P[:,1,:] - P[:,0,:]
    dv = P[:,2,:] - P[:,0,:]
    duu = np.sum(du**2,axis=1)
    dvv = np.sum(dv**2,axis=1)
    duv = np.sum(du*dv,axis=1)
    W = np.zeros((len(tri.simplices)))
    H = np.zeros((len(tri.simplices),2), dtype=np.float64)
    for i in range(len(tri.simplices)):
        A = np.array([[duu[i],duv[i]],[duv[i],dvv[i]]])
        w,v = eig(A)
        H[i] = np.sort(w)[::-1]
        W[i] = np.linalg.det(A)
    return H, W

H, W = curvature(tri, points)

K = W / (H[:,0] + H[:,1])**2

mean_K = np.zeros(len(points))
for i in range(len(points)):
    neighbors = tri.vertex_neighbor_vertices[1] == i
    if not np.any(neighbors):
        continue # skip vertices without neighbors
    neighbor_indices = tri.vertex_neighbor_vertices[0][np.nonzero(neighbors)]
    mean_K[i] = np.mean(K[neighbor_indices])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], c=mean_K, cmap=plt.cm.coolwarm)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()


In [ ]:
# fit a second-degree polynomial to the point cloud
z = points[:, 2]
x = points[:, 0]
y = points[:, 1]
poly_coeffs = np.polyfit2d(x, y, z, deg=2)

# evaluate the polynomial on a grid to visualize the fitted surface
X, Y = np.meshgrid(np.linspace(np.min(x), np.max(x), 100), np.linspace(np.min(y), np.max(y), 100))
Z = poly_coeffs[0]*X**2 + poly_coeffs[1]*Y**2 + poly_coeffs[2]*X*Y + poly_coeffs[3]*X + poly_coeffs[4]*Y + poly_coeffs[5]

# plot the point cloud and the fitted surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c=mean_K, cmap=plt.cm.coolwarm)
ax.plot_surface(X, Y, Z, alpha=0.5)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()
